In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evoman.environment import Environment
from controller_custom import player_controller
from multiprocessing import Pool
from scipy.stats import ttest_ind, mannwhitneyu
import pandas as pd
import warnings
import itertools
import pandas as pd

In [5]:
def run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group, elitism_count=1):
	"""
	Runs Genetic Algorithm A with the provided environment and parameters.

	Parameters:
	- env: Evoman Environment instance.
	- npopulation: Population size.
	- gens: Number of generations.
	- mutation_rate: Mutation rate.
	- dom_u: Upper bound for gene values.
	- dom_l: Lower bound for gene values.
	- elitism_count: Number of elites to preserve.

	Returns:
	- history_mean: List of mean fitness per generation.
	- history_max: List of max fitness per generation.
	- best_solution: Best solution found.
	"""
	
	n_hidden_neurons = 10
	n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
	
	# Run the simulation and return the fitness
	def simulate(x):
		env.player_controller.set(x, env.get_num_sensors())
		f, _, _, _ = env.play(pcont=x)
		return f
	
	# Evaluate the current population
	def evaluate(population):
		return np.array([simulate(individual) for individual in population])

	# Tournament Selection
	def tournament_selection(population, fitness, k=5):
		selected = []
		for _ in range(len(population)):
			contenders = np.random.choice(len(population), k, replace=False)
			winner = contenders[np.argmax(fitness[contenders])]
			selected.append(population[winner])
		return np.array(selected)

	# Uniform Crossover
	def crossover(parent1, parent2):
		mask = np.random.rand(n_vars) < 0.5
		child1 = np.where(mask, parent1, parent2)
		child2 = np.where(mask, parent2, parent1)
		return child1, child2

	# Gaussian Mutation
	def mutate(child):
		for i in range(n_vars):
			if np.random.rand() < mutation_rate:
				child[i] += np.random.normal(0, 0.1)
				child[i] = np.clip(child[i], dom_l, dom_u)
		return child

	# Initialize population
	population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
	fitness = evaluate(population)

	# Record fitness over generations
	history_mean = []
	history_max = []

	# Genetic Algorithm Loop
	for generation in range(1, gens + 1):
		# Selection
		selected = tournament_selection(population, fitness)

		# Crossover
		offspring = []
		for i in range(0, npopulation, 2):
			parent1, parent2 = selected[i], selected[i+1]
			child1, child2 = crossover(parent1, parent2)
			offspring.extend([child1, child2])
		offspring = np.array(offspring)[:npopulation]

		# Mutation
		offspring = np.array([mutate(child) for child in offspring])

		# Evaluation
		offspring_fitness = evaluate(offspring)

		# Replacement: Elitism (preserve top elitism_count individuals)
		elite_indices = np.argsort(fitness)[-elitism_count:]  
		elite_individuals = population[elite_indices]  
		elite_fitness = fitness[elite_indices]  
		worst_indices = np.argsort(offspring_fitness)[:elitism_count]  
		offspring[worst_indices] = elite_individuals  
		offspring_fitness[worst_indices] = elite_fitness  # Replace the fitness of the worst with elite fitness

		# Update population and fitness
		population, fitness = offspring, offspring_fitness

		# Record statistics
		history_mean.append(np.mean(fitness))
		history_max.append(np.max(fitness))

		# Logging
		print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}')

	# Get best solution
	best_idx = np.argmax(fitness)
	best_solution = population[best_idx]

	np.savetxt(os.path.join('results', f'best_solution_A_{enemy_group}.txt'), best_solution)

	return history_mean, history_max, best_solution


In [9]:
num_runs = 10
npopulation = 200
enemy_group_1 = [1, 4, 5, 8]
enemy_group_2 = [2, 3, 6, 7]

# Define the hyperparameter grid
param_grid = {
    'npopulation': [100, 200],
    'gens': [10, 30],
    'mutation_rate': [0.01, 0.05],
    'dom_u': [1],
    'dom_l': [-1],
    'elitism_count': [1, 2]
}

# Create a list of all parameter combinations
keys, values = zip(*param_grid.items())
param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Initialize data storage for results
results = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Generation': [],
    'Mean Fitness': [],
    'Max Fitness': [],
    'Best Solution': [],
    'npopulation': [],
    'gens': [],
    'mutation_rate': [],
    'elitism_count': []
}

best_solutions = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Best Solution': [],
    'npopulation': [],
    'gens': [],
    'mutation_rate': [],
    'elitism_count': []
}

# Iterate over each combination of hyperparameters
for params in param_combinations:
    print(f"Running with params: {params}")
    
    # Repeat for the number of runs
    for run in range(1, num_runs + 1):
        print(f"Run {run}")

        experiment_name = f'EA1_EG1_run{run}_npop{params["npopulation"]}_gens{params["gens"]}_mut{params["mutation_rate"]}'
        os.makedirs(experiment_name, exist_ok=True)

        env = Environment(experiment_name=experiment_name,
                          multiplemode="yes",
                          enemies=enemy_group_1,
                          playermode="ai",
                          player_controller=player_controller(n_hidden_neurons),
                          enemymode="static",
                          level=2,
                          speed="fastest",
                          visuals=False)

        mean, max, best = run_algorithm_a(env, 
                                          params['npopulation'], 
                                          params['gens'], 
                                          params['mutation_rate'], 
                                          params['dom_u'], 
                                          params['dom_l'], 
                                          enemy_group='enemy_group_1')

        for gen in range(params['gens']):
            results['Algorithm'].append('A')
            results['Enemy Group'].append(1)
            results['Run'].append(run)
            results['Generation'].append(gen)
            results['Mean Fitness'].append(mean[gen])
            results['Max Fitness'].append(max[gen])
            results['Best Solution'].append(best)
            results['npopulation'].append(params['npopulation'])
            results['gens'].append(params['gens'])
            results['mutation_rate'].append(params['mutation_rate'])
            results['elitism_count'].append(params['elitism_count'])

        best_solutions['Algorithm'].append('A')
        best_solutions['Enemy Group'].append(1)
        best_solutions['Run'].append(run)
        best_solutions['Best Solution'].append(best)
        best_solutions['npopulation'].append(params['npopulation'])
        best_solutions['gens'].append(params['gens'])
        best_solutions['mutation_rate'].append(params['mutation_rate'])
        best_solutions['elitism_count'].append(params['elitism_count'])

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', 'grid_search_results.csv'), index=False)

best_solutions_df = pd.DataFrame(best_solutions)
best_solutions_df.to_csv(os.path.join('results', 'grid_search_best_solutions.csv'), index=False)


Running with params: {'npopulation': 100, 'gens': 10, 'mutation_rate': 0.01, 'dom_u': 1, 'dom_l': -1, 'elitism_count': 1}
Run 1

MESSAGE: Pygame initialized for simulation.
